<h1>Context analysis</h1>

This notebook will explore the context of the app usage. This will be done by:
<ul>
    <li>Identifying how many permissions each app is requesting</li>
    <li>Identify how many permissions are approved for each app</li>
    <li>Identifying which apps request the same permission but have different response</li>
</ul>
    
    

<h2>1. Importing modules </h2>

Pandas is required in order to manage the dataframe
Numpy is used to handle numpy arrays
os and path are used to access particular files

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("context.csv")
df = df.drop(columns = ['Unnamed: 3'])
df.columns = ["app", "permission", "response"]
print(df)

                          app  \
0     Qualcomm MobileSecurity   
1     Qualcomm MobileSecurity   
2                     YouTube   
3                     YouTube   
4                     YouTube   
...                       ...   
3522            Android Setup   
3523            Android Setup   
3524            Android Setup   
3525            Android Setup   
3526            Android Setup   

                                             permission  response  
0               android.permission.ACCESS_NETWORK_STATE      True  
1             android.permission.RECEIVE_BOOT_COMPLETED      True  
2                      android.permission.READ_CONTACTS     False  
3                       android.permission.GET_ACCOUNTS     False  
4      com.sonyericsson.home.permission.BROADCAST_BADGE     False  
...                                                 ...       ...  
3522         android.permission.GET_ACCOUNTS_PRIVILEGED      True  
3523                  android.permission.WRITE_CONTACTS    

In [3]:
appPerms = pd.DataFrame(columns = ["App", "Perms"])

for app in df.app.unique():
    appDF = df[df["app"] == app]
    lenAppPerm = len(appPerms)
    appPerms.at[lenAppPerm, "App"] = app
    appPerms.at[lenAppPerm, "Perms"] = len(appDF)

print(appPerms)

                            App Perms
0       Qualcomm MobileSecurity     2
1                       YouTube    33
2      Android Services Library     1
3    Phone and MessagingStorage     7
4                        Google    98
..                          ...   ...
187          com.qualcomm.embms     3
188            Contacts Storage    15
189                  MapPartner     3
190          CaptivePortalLogin     5
191                      Gboard    21

[192 rows x 2 columns]


In [4]:
appPermsResponse = pd.DataFrame(columns = ["App", "PermsApproved", "PermsRejected"])

apps = df.app.unique()

for app in apps:
    appDF = df[df["app"] == app]
    responseDF = appDF[appDF["response"] == True]
    lenAppPerm = len(appPermsResponse)
    appPermsResponse.at[lenAppPerm, "App"] = app
    appPermsResponse.at[lenAppPerm, "PermsApproved"] = len(responseDF)
    appPermsResponse.at[lenAppPerm, "PermsRejected"] = len(appDF) - len(responseDF)

print(appPermsResponse)

                            App PermsApproved PermsRejected
0       Qualcomm MobileSecurity             2             0
1                       YouTube            17            16
2      Android Services Library             1             0
3    Phone and MessagingStorage             7             0
4                        Google            83            15
..                          ...           ...           ...
187          com.qualcomm.embms             3             0
188            Contacts Storage            15             0
189                  MapPartner             3             0
190          CaptivePortalLogin             5             0
191                      Gboard            16             5

[192 rows x 3 columns]


In [5]:
permsDF = pd.DataFrame(columns = ["permission", "rejected", "rApps", "accepted", "aApps"])

permissions = df.permission.unique()
print(len(permissions))


for permission in permissions:
    permDF = df[df.permission == permission]
    lenPerms = len(permsDF)
    permsDF.at[lenPerms,"permission"] = permission
    permsDF.at[lenPerms,"rejected"] = len(permDF[permDF["response"] == False])
    permsDF.at[lenPerms,"rApps"] = permDF[permDF["response"] == False].app.values
    permsDF.at[lenPerms,"accepted"] = len(permDF[permDF["response"] == True])
    permsDF.at[lenPerms,"aApps"] = permDF[permDF["response"] == True].app.values    
    
for i, row in permsDF.iterrows():
    print(row)


752
permission              android.permission.ACCESS_NETWORK_STATE
rejected                                                      0
rApps                                                        []
accepted                                                    110
aApps         [Qualcomm MobileSecurity, YouTube, Google, com...
Name: 0, dtype: object
permission            android.permission.RECEIVE_BOOT_COMPLETED
rejected                                                      0
rApps                                                        []
accepted                                                    115
aApps         [Qualcomm MobileSecurity, YouTube, Google, com...
Name: 1, dtype: object
permission                     android.permission.READ_CONTACTS
rejected                                                     18
rApps         [YouTube, Google Play Store, Google Indic Keyb...
accepted                                                     28
aApps         [Google, Telegram, WhatsApp, Messages, C